In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

import pymorphy2
import nltk

In [2]:
import util

In [3]:
data = pd.read_csv('feedback.csv/X_train.csv')
data.shape

(15587, 11)

In [4]:
data.columns

Index(['sku', 'categoryLevel1Id', 'categoryLevel2Id', 'brandId', 'property',
       'userName', 'reting', 'date', 'comment', 'commentNegative',
       'commentPositive'],
      dtype='object')

In [5]:
data.sku.value_counts().describe()

count    2698.000000
mean        5.777242
std         5.986300
min         1.000000
25%         1.000000
50%         3.000000
75%         8.000000
max        25.000000
Name: sku, dtype: float64

In [6]:
texts = util.prepare_texts(data.comment, bigrams = False)

In [56]:
for i, t in texts.sample(3).items():
    print(data.comment[i])
    print(t)
    print()

Плита полностью отрабатывает свои деньги.Готовлю много.Пеку.Дешево, но сердито.Столкнулись с гарантийным ремонтом. Ребята быстро отреагировали.Всем советую
плита полностью отрабатывать свой деньги.готовить много.пеку.дешево , но сердито.столкнуться с гарантийный ремонт . ребята быстро отреагировали.всем советовать

Искали простенькую и недорогую вытяжку для кухни, привлекла эта. Она маленькая и компактная, легко встраивается. Моется тоже легко. для нашей маленькой кухни её вполне хватает. тянет воздух отлично, теперь на кухне не душно во время готовки, не пахнет жареным луком и другими продуктами. Производительность хорошая, при этом вытяжка совсем не шумная. Удобное управление, 3 скорости, можно выбрать любую в зависимости от желаемой мощности. Освещает плиту хорошо.
искать простенький и недорогой вытяжка для кухня , привлечь этот . она маленький и компактный , легко встраиваться . мыться тоже легко . для наш маленькая кухня её вполне хватать . тянуть воздух отлично , теперь на кухня 

In [12]:
texts_by_sku = texts.groupby(data.sku).aggregate(lambda x: ' '.join(x))
texts_by_sku.shape

(2698,)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
from nltk.corpus import stopwords
stopWords = stopwords.words('russian')

tf_vectorizer = CountVectorizer(max_df=0.5, min_df=10,
                                stop_words=stopWords)

In [27]:
tf_vectorizer.fit(texts)
print(len(tf_vectorizer.vocabulary_))

4654


In [28]:
counts = tf_vectorizer.transform(texts_by_sku)
counts.shape

(2698, 4654)

In [37]:
counts_names = tf_vectorizer.get_feature_names()

In [31]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
lda = LatentDirichletAllocation( max_iter=10, n_topics=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    n_jobs = -1,
                                    verbose = 1,
                                    random_state=0)

In [69]:
def get_top_words(model, feature_names, n_top_words):
    tops = []
    for topic_idx, topic in enumerate(model.components_):
        tops.append(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    return tops

In [46]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [52]:
nmf = NMF(n_components=40, random_state=1, init = 'nndsvdar',
          alpha=.1, l1_ratio=.5, max_iter = 30)

In [53]:
nmf.fit(counts)

NMF(alpha=0.1, beta=1, eta=0.1, init='nndsvdar', l1_ratio=0.5, max_iter=30,
  n_components=40, nls_max_iter=2000, random_state=1, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [70]:
get_top_words(nmf, counts_names, 10)

['весь советовать отличный ничто пока общий модель супер очень отлично',
 'пылесос уборка фильтр мощность пыль контейнер ковёр мыть щётка мешок',
 'год работать пользоваться сломаться новый месяц назад отличный нарекание покупать',
 'чайник крышка вода дизайн пластик кипятить вод запах это шуметь',
 'волос выпрямитель очень выпрямлять нагреваться щипец пользоваться быстро температура локон',
 'машинка стирка стирать машина отжим бельё барабан программа режим время',
 'телефон камера экран очень кнопка день звук _brackets_positive динамик удобный',
 'ноутбук ноут проблема работать это экран игра очень минус диск',
 'телевизор картинка изображение тв звук канал качество фильм смотреть usb',
 'вода вод это горячий водонагреватель время отключение весь мыть уборка',
 'плеер видео звук кнопка наушник экран музыка работать файл часы',
 'сок жмых соковыжималка яблоко сухой очень купить выжимать стакан легко',
 'работать блок комната кондиционер вентилятор модель внешний режим шум охлаждать',


In [71]:
nmf_top = get_top_words(nmf, counts_names, 3)

In [60]:
nmf_tran = nmf.transform(tf_vectorizer.transform(texts))

In [68]:
pred_topics = np.argmax(nmf_tran, axis = 1)

In [ ]:
def get_top_words()

In [80]:
for i, text in data.comment.sample(1).items():
    print(text)
    topic = pred_topics[i]
    print(topic, nmf_top[topic])

Берите не пожалеете... звук отличный.. плеер супер.. работает сразу с несколькими приложениями, нет тормозов..минусов у него нет
10 плеер видео звук


In [55]:
lda.fit(counts)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=-1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=1)

In [57]:
print_top_words(lda, counts_names, 10)

Topic #0: игра телефон ноутбук весь ноут это экран играть память _period
Topic #1: очень год весь вода это купить чайник машинка работать утюг
Topic #2: пылесос очень волос насадка фильтр год пользоваться это мощность щётка
Topic #3: очень весь это пользоваться купить год довольный просто плита _many_exclamations
Topic #4: весь очень это хороший купить _period качество работать отличный звук



In [644]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import timeit
import datetime
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nile.api.v1 import clusters
from nile.api.v1 import aggregators as na
from qb2.api.v1 import filters as qf
from nile.api.v1 import filters as nf
from nile.api.v1 import extractors as ne
import ast
from collections import defaultdict
from keras.preprocessing import sequence
import time
from gensim.models import word2vec
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers.advanced_activations import PReLU
import matplotlib
import pickle
import gc
from keras.models import load_model
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from __future__ import print_function
from time import time
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from tqdm import tqdm
import pymorphy2

In [ ]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))

for n_topics in [100]:
    lda = LatentDirichletAllocation( max_iter=40,n_topics=n_topics,
                                    learning_method=None,
                                    learning_offset=50.,
                                    random_state=0)#,n_jobs = 16)
    tf = tf_vectorizer.fit_transform(res_2)
    t0 = time()
    lda.fit(tf)
    print("done in %0.3fs." % (time() - t0))
    print("\nTopics in LDA model:")
    tf_feature_names = tf_vectorizer.get_feature_names()
    print_top_words(lda, tf_feature_names, n_top_words)
    print('-------------------------------------------------------------------------')
    print('-------------------------------------------------------------------------')
    print('-------------------------------------------------------------------------')
    print('-------------------------------------------------------------------------')

Fitting LDA models with tf features, n_samples=2000 and n_features=20000...
